In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

## Dados escolares

In [10]:
#abrindo os dados
dados_escolares = pd.read_csv('DadosEscolares1996-2015.csv', encoding = 'latin1')
nota = pd.read_csv('ENEM2015.csv')
#agrupando por escolas
dados_escolares = dados_escolares.groupby('CODMEC').mean()
#Apagando a classe desbalanceada
nota = nota[nota.Classificacao != 0]
#Em todos Dataframes temos mais escolas do que o nosso target
dados_escolares = dados_escolares.reset_index()[dados_escolares.reset_index().CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
dados_escolares.drop(dados_escolares.columns[dados_escolares.isnull().mean() != 0], axis=1, inplace = True)
#Alterando o nome da coluna para o merge
nota.rename(columns = {'CODIGO DA ENTIDADE':'CODMEC'}, inplace = True)
#criando o merge com base na coluna CODMEC usando o dataframe target e os dados escolares
new_df = pd.merge(dados_escolares, nota, on = 'CODMEC')
new_df.drop(['CODMEC', 'ANO','NUMERO DE PARTICIPANTES'], axis = 1, inplace = True)
#criando o data e o target
data = new_df.drop('Classificacao', axis = 1)
target = new_df.Classificacao

C:\Users\giova\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (6,8,11,17,18,19,20,111,240) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
melhor_score = 0
melhores_colunas = []
while len(data.columns) >= 10:
    lm = LinearRegression()
    scaler = StandardScaler()
    data1 = scaler.fit_transform(data)
    coefs = [0 for x in range(len(data.columns))]
    kf = KFold(5, shuffle = True)
    r = []
    for train_i, test_i in kf.split(data):
        x_train, x_test = data1[train_i], data1[test_i]
        y_train, y_test = target[train_i], target[test_i]
        lm.fit(x_train, y_train)
        r.append(lm.score(x_test, y_test))
        for x in range(len(coefs)):
            coefs[x] = coefs[x] + abs(lm.coef_[x])
    print('Score:',sum(r)/5)
    if melhor_score< sum(r)/5:
        melhor_score = sum(r)/5
        melhores_colunas = list(data.columns)
    ind = coefs.index(min(coefs))
    print('Smaller coef columns:', data.columns[ind], min(coefs)/5)
    data.drop(data.columns[ind], axis=1, inplace = True)

Score: -2941.875232263642
Smaller coef columns: EE_Down_EP 0.0
Score: -1.2810464535349053
Smaller coef columns: EE_Ret_EP 0.0
Score: -2.502989457551078e+24
Smaller coef columns: EE_TDI_EP 0.0
Score: -0.7160637380038658
Smaller coef columns: EE_CON_EP 0.0
Score: -2.4158184546372614
Smaller coef columns: EE_OUT_EP 0.0
Score: -1.6608606677115934
Smaller coef columns: EE_CON_PRE 0.0
Score: -830.26962787703
Smaller coef columns: EE_OUT_PRE 0.0
Score: -3.780709439242161
Smaller coef columns: EE_EJA 0.0
Score: -41.414965649395214
Smaller coef columns: EE_EP 0.0
Score: -3.009975780855148e+25
Smaller coef columns: SITUAÃâ¡ÃÆO 5.996359689622621e-12
Score: -5.819653124537604e+20
Smaller coef columns: EE_BAUD_CRE 0.00819690143971244
Score: -3.492059024835135e+21
Smaller coef columns: EE_VIS_CRE 0.006447786671520066
Score: -2.667554454727227e+18
Smaller coef columns: EE_Transtornos_MED 0.005697762976977392
Score: -1.0817831473883683
Smaller coef columns: EE_CON_EJA 1.3420375921668892e-13
Score:

Score: -1.9279271947720016
Smaller coef columns: EE_AUD_CRE 0.032436303271377365
Score: 0.1376913021900451
Smaller coef columns: EE_Asperger_MED 0.03842025031707898
Score: -0.023481951713061177
Smaller coef columns: >6 0.028287356934961715
Score: 0.33904418563853495
Smaller coef columns: EE_Asperger_FUND 0.0330997616312288
Score: 0.2531239682981243
Smaller coef columns: TOT_EE_MED 0.028037257956430084
Score: -2.689572604782646
Smaller coef columns: 2 0.022685131605676208
Score: -0.18333838673003483
Smaller coef columns: ALES1A4_SEMI 0.03987239482750969
Score: -0.2379352673387781
Smaller coef columns: ALESMED_SEMI 0.023715576783665947
Score: 0.02560339188450157
Smaller coef columns: EE_Surdo_Cegueira_MED 0.04209291992206286
Score: 0.11809912791053201
Smaller coef columns: TOT_EE_PRE 0.04515077775570621
Score: 0.17113096127604252
Smaller coef columns: TOT_EE_FUND 0.04688053852821802
Score: 0.11708919595775957
Smaller coef columns: TOT_EE 0.02980429015833338
Score: -0.11873647880107943
Sm

In [12]:
print('O melhor score foi: ', melhor_score)
print('Com as seguintes colunas: \n', melhores_colunas)

O melhor score foi:  0.45967621189926744
Com as seguintes colunas: 
 ['Qtde_Docentes', 'MENOR3', '3', '0A3', '0A4', '4A6', '5A6', 'TotalEdInf', 'ALEM1S', 'ALEM2S', 'CLEF1S', 'CLEF4S', 'CLEF5S', 'ALEF1S', 'ALEF4S', 'ALEF5S', 'ALEF7S', 'ALEF8S', 'CLEF1A4', 'CLEF5A8', 'CLEF1A8', 'ALEF1A4', 'ALEF5A8', 'ALEF1A8', 'CLE9F1A5', 'CLE9F6A9', 'ALE9F6A9', 'CLE9F1A9', 'ALE9F1A9', 'CLFUNDI', 'ALFUNDI', 'CLFUNDII', 'ALFUNDII', 'CLFUND', 'ALFUND', 'TOTAL_ALUNOS']


## ESC_RMSP

In [63]:
#abrindo os dados
esc_rmsp = pd.read_csv('ESC2013_RMSP_CEM.csv', encoding = 'latin1')
#agrupando por escolas
esc_rmsp = esc_rmsp.groupby('CODESC').mean()
#Apagando a classe desbalanceada
#Em todos Dataframes temos mais escolas do que o nosso target
esc_rmsp = esc_rmsp.reset_index()[esc_rmsp.reset_index().CODESC.isin(nota['CODMEC'])]
esc_rmsp.rename(columns = {'CODESC': 'CODMEC'}, inplace = True)
#Dataframe com bastante valores NaN, limpando eles
esc_rmsp.drop(esc_rmsp.columns[esc_rmsp.isnull().mean() >= 0.15], axis=1, inplace = True)
esc_rmsp.dropna(inplace = True, how = 'any')
#criando o merge com base na coluna CODMEC usando o dataframe target e os dados escolares
new_df = pd.merge(esc_rmsp, nota, on = 'CODMEC')
new_df.drop(['CODMEC','NUMERO DE PARTICIPANTES'], axis = 1, inplace = True)
print(new_df.shape)
#criando o data e o target
data = new_df.drop('Classificacao', axis = 1)
print(data.shape)
target = new_df.Classificacao
data = data.astype('float64')

(931, 52)
(931, 51)


In [64]:
melhor_score = 0
melhores_colunas = []
while len(data.columns) >= 10:
    lm = LinearRegression()
    scaler = StandardScaler()
    data1 = scaler.fit_transform(data)
    coefs = [0 for x in range(len(data.columns))]
    kf = KFold(5, shuffle = True)
    r = []
    for train_i, test_i in kf.split(data):
        x_train, x_test = data1[train_i], data1[test_i]
        y_train, y_test = target[train_i], target[test_i]
        lm.fit(x_train, y_train)
        r.append(lm.score(x_test, y_test))
        for x in range(len(coefs)):
            coefs[x] = coefs[x] + abs(lm.coef_[x])
    print('Score:',sum(r)/5)
    if melhor_score< sum(r)/5:
        melhor_score = sum(r)/5
        melhores_colunas = list(data.columns)
    ind = coefs.index(min(coefs))
    print('Smaller coef columns:', data.columns[ind], min(coefs)/5)
    data.drop(data.columns[ind], axis=1, inplace = True)

Score: -2.734628497313278e+24
Smaller coef columns: REG_MED 0.005177498010476417
Score: -1.8227853526153085e+24
Smaller coef columns: ESP_FU_8A 0.0023403293751863784
Score: -5.153473975117186e+23
Smaller coef columns: APR3EM_11 0.005854371324113572
Score: -6.696443451129497e+23
Smaller coef columns: CODESCTX 0.005829622112760743
Score: -1.0840565306462974e+23
Smaller coef columns: REG_MED_I 0.006459124739890317
Score: -1.3613548004881747e+23
Smaller coef columns: ESP_IN_PE 0.0033732015796161426
Score: 0.5066345989451853
Smaller coef columns: EJA_PROJ 9.821677915411442e-10
Score: -1.8379470892043602e+21
Smaller coef columns: REG_MED_P 0.007022974588934937
Score: 0.497844510719421
Smaller coef columns: ESP_EJA_M 1.0315659437765134e-09
Score: 0.507666567154349
Smaller coef columns: REG_IN_PE 0.005937784465316421
Score: 0.5070723559661674
Smaller coef columns: ESP_EJA_F 1.889979728275648e-10
Score: 0.5033115678097013
Smaller coef columns: ESP_MED_I 1.1742693928340487e-09
Score: -3.03315560

In [65]:
print('O melhor score foi: ', melhor_score)
print('Com as seguintes colunas: \n', melhores_colunas)

O melhor score foi:  0.5278200238739648
Com as seguintes colunas: 
 ['CODMUN', 'CODDIST', 'BIBLIO', 'REG_IN_CR', 'ESP_FU_9A', 'EJA', 'EJA_FUND', 'EJA_MED', 'AB1EM_10', 'AP9EF_12', 'AP9EF_13', 'AP3EM_14', 'INSE_AB']


## Rendimento

In [80]:
#abrindo os dados
rendimento = pd.read_csv('RendimentoEscolar2000-2015.csv', encoding = 'latin1')
#agrupando por escolas
rendimento = rendimento.groupby('CODMEC').mean()
#Em todos Dataframes temos mais escolas do que o nosso target
rendimento = rendimento.reset_index()[rendimento.reset_index().CODMEC.isin(nota['CODMEC'])]
def t(x):
    if x==0:
        return None
    else:
        return x
for c in rendimento.columns:
    rendimento[c] = rendimento[c].apply(t)
rendimento.drop(rendimento.columns[rendimento.isnull().mean() >= 0.15], axis=1, inplace = True)
rendimento.dropna(inplace = True, how = 'any')
#criando o merge com base na coluna CODMEC usando o dataframe target e os dados escolares
new_df = pd.merge(rendimento, nota, on = 'CODMEC')
new_df.drop(['CODMEC','NUMERO DE PARTICIPANTES'], axis = 1, inplace = True)
#criando o data e o target
data = new_df.drop('Classificacao', axis = 1)
target = new_df.Classificacao

C:\Users\giova\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [83]:
melhor_score = 0
melhores_colunas = []
while len(data.columns) >= 10:
    lm = LinearRegression()
    scaler = StandardScaler()
    data1 = scaler.fit_transform(data)
    coefs = [0 for x in range(len(data.columns))]
    kf = KFold(5, shuffle = True)
    r = []
    for train_i, test_i in kf.split(data):
        x_train, x_test = data1[train_i], data1[test_i]
        y_train, y_test = target[train_i], target[test_i]
        lm.fit(x_train, y_train)
        r.append(lm.score(x_test, y_test))
        for x in range(len(coefs)):
            coefs[x] = coefs[x] + abs(lm.coef_[x])
    print('Score:',sum(r)/5)
    if melhor_score< sum(r)/5:
        melhor_score = sum(r)/5
        melhores_colunas = list(data.columns)
    ind = coefs.index(min(coefs))
    print('Smaller coef columns:', data.columns[ind], min(coefs)/5)
    data.drop(data.columns[ind], axis=1, inplace = True)

Score: 0.29802150912750164
Smaller coef columns: TOTAL_EM_REPROV 0.05347996366973018
Score: 0.14860557194308027
Smaller coef columns: TOTALEF_REPROV 0.0894921784800943
Score: 0.16700441293622142
Smaller coef columns: EFAPROV9ANOS6S 0.065063817037156
Score: 0.2811839242852897
Smaller coef columns: EFAPROV9ANOS1S 0.05639434451218146
Score: 0.27558743131501223
Smaller coef columns: EFAPROV9ANOS3S 0.057832983769928015
Score: 0.2516509126276989
Smaller coef columns: EFAPROV9ANOS5S 0.053239793846330874
Score: 0.23434215861098856
Smaller coef columns: EFAPROV9ANOS7S 0.103762595347593
Score: 0.30333303723678534
Smaller coef columns: EFAPROV9ANOS4S 0.10821195212337413


In [84]:
print('O melhor score foi: ', melhor_score)
print('Com as seguintes colunas: \n', melhores_colunas)

O melhor score foi:  0.30333303723678534
Com as seguintes colunas: 
 ['ANO', 'EFAPROV9ANOS2S', 'EFAPROV9ANOS4S', 'EFAPROV9ANOS8S', 'EFAPROV9ANOS9S', 'TOTALEF_APROV', 'EMAPROV1S', 'EMAPROV2S', 'EMAPROV3S', 'TOTAL_EM_APROV']
